# 1. Data Gathering 

In [1]:
import numpy as np
import pandas as pd
import ast # for converting string to list 

In [2]:
movies = pd.read_csv("tmdb_5000_movies.csv")
credits = pd.read_csv("tmdb_5000_credits.csv")

In [3]:
movies.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800


In [4]:
credits.head(1)

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [5]:
# credits.head(1)['crew'].values

## Merging both dataframes

we can merge both dataset on the basis of either title or id

In [6]:
movies.shape

(4803, 20)

In [7]:
credits.shape
 

(4803, 4)

In [8]:
movies = movies.merge(credits,on='title')
movies.shape

(4809, 23)

# Preprocessing 

we will be creating a content based recommender system so we will be using only that columns which is helping us in creaditing tags so we are going to ask question before selecting a columns that :
    **Where this column is helpfull in creating tags ?** :
        

In [9]:
movies.head(1).columns

Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count', 'movie_id', 'cast', 'crew'],
      dtype='object')

### columns we will be kepping 
* genres
* id
* keywords
* overview 
* title
* cast 
* crew
* release date





In [10]:
movies = movies[['movie_id','title','genres','keywords', 'overview','cast','crew']]

In [11]:
# movies.info()

**we will be creating new data frame from this dataset having three columns :**
* movie id
* title
* tag 

now we need to creat this tag columns

In [12]:
# Missing data

In [13]:
# movies[movies.isnull()==True]
movies.isnull().sum()

movie_id    0
title       0
genres      0
keywords    0
overview    3
cast        0
crew        0
dtype: int64

there is 3 movies in which overoview is missing, since 3 is not big number we will drop this

In [14]:
movies.dropna(inplace=True)

In [15]:
movies.duplicated().sum()

0

In [16]:
movies.iloc[0]['genres']

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

it is list of dictionary we need to bring all keywords into a list 
for example :
    ['action','adventure','fantasy','scify']

In [17]:
def convert(obj):
    L=[]
    for i in ast.literal_eval(obj):
        L.append(i['name'])
    return L
        

In [18]:
movies['genres']=movies.genres.apply(convert)

In [19]:
movies['keywords']=movies.keywords.apply(convert)

In [20]:
movies.head(1)

,movie_id,title,genres,keywords,overview,cast,crew
0,19995,Avatar,"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","In the 22nd century, a paraplegic Marine is di...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [21]:
def convert_cast(obj):
    count=0
    l=[]
    for j in ast.literal_eval(obj):
        if count!=3:
            l.append(j['name'])
            count+=1
        else:
            break
    return l

In [22]:
movies['cast']=movies['cast'].apply(convert_cast)

In [23]:
 movies.head(1)

,movie_id,title,genres,keywords,overview,cast,crew
0,19995,Avatar,"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","In the 22nd century, a paraplegic Marine is di...","[Sam Worthington, Zoe Saldana, Sigourney Weaver]","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [24]:
def convert_crew(obj):
    director=[]
    for i in ast.literal_eval(obj):
        if i['job']=='Director':
            director.append(i['name'])
    return director

In [25]:
movies['crew']=movies['crew'].apply(convert_crew)

In [26]:
movies.head(1)

,movie_id,title,genres,keywords,overview,cast,crew
0,19995,Avatar,"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","In the 22nd century, a paraplegic Marine is di...","[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[James Cameron]


In [27]:
movies['overview']=movies['overview'].apply(lambda x:x.split()) #for i in movies['overview']: print(i.split(" ")) same code

now concatenating genres , keywords , overview and crew , cast columns into one single list then converting that list into one string

In [28]:
def transform(obj):
    l = []
    for i in obj:
        l.append("".join(i.split(" ")))
    return l


# you can use this alsp movies.apply() lambda x:[i.replace(" ".'') for i in x]

In [29]:
movies['cast']=movies['cast'].apply(transform)
movies['crew']=movies['crew'].apply(transform)
movies['genres']=movies['genres'].apply(transform)
movies['keywords']=movies['keywords'].apply(transform)
movies['overview']=movies['overview'].apply(transform)

In [30]:
movies.head(5)

,movie_id,title,genres,keywords,overview,cast,crew
0,19995,Avatar,"[Action, Adventure, Fantasy, ScienceFiction]","[cultureclash, future, spacewar, spacecolony, ...","[In, the, 22nd, century,, a, paraplegic, Marin...","[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron]
1,285,Pirates of the Caribbean: At World's End,"[Adventure, Fantasy, Action]","[ocean, drugabuse, exoticisland, eastindiatrad...","[Captain, Barbossa,, long, believed, to, be, d...","[JohnnyDepp, OrlandoBloom, KeiraKnightley]",[GoreVerbinski]
2,206647,Spectre,"[Action, Adventure, Crime]","[spy, basedonnovel, secretagent, sequel, mi6, ...","[A, cryptic, message, from, Bond’s, past, send...","[DanielCraig, ChristophWaltz, LéaSeydoux]",[SamMendes]
3,49026,The Dark Knight Rises,"[Action, Crime, Drama, Thriller]","[dccomics, crimefighter, terrorist, secretiden...","[Following, the, death, of, District, Attorney...","[ChristianBale, MichaelCaine, GaryOldman]",[ChristopherNolan]
4,49529,John Carter,"[Action, Adventure, ScienceFiction]","[basedonnovel, mars, medallion, spacetravel, p...","[John, Carter, is, a, war-weary,, former, mili...","[TaylorKitsch, LynnCollins, SamanthaMorton]",[AndrewStanton]


In [31]:
movies['tags'] = movies.keywords + movies.genres + movies.overview + movies.cast + movies.crew

In [32]:
new_df = movies[['movie_id','title','tags']]


In [33]:
new_df['tags']=new_df['tags'].apply(lambda x:" ".join(x))

C:\Users\rani\AppData\Local\Temp\ipykernel_16476\487797088.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags']=new_df['tags'].apply(lambda x:" ".join(x))


In [34]:
new_df

,movie_id,title,tags
0,19995,Avatar,cultureclash future spacewar spacecolony socie...
1,285,Pirates of the Caribbean: At World's End,ocean drugabuse exoticisland eastindiatradingc...
2,206647,Spectre,spy basedonnovel secretagent sequel mi6 britis...
3,49026,The Dark Knight Rises,dccomics crimefighter terrorist secretidentity...
4,49529,John Carter,basedonnovel mars medallion spacetravel prince...
...,...,...,...
4804,9367,El Mariachi,unitedstates–mexicobarrier legs arms paperknif...
4805,72766,Newlyweds,Comedy Romance A newlywed couple's honeymoon i...
4806,231617,"Signed, Sealed, Delivered",date loveatfirstsight narration investigation ...
4807,126186,Shanghai Calling,When ambitious New York attorney Sam is sent t...


In [35]:
new_df['tags'][0]

'cultureclash future spacewar spacecolony society spacetravel futuristic romance space alien tribe alienplanet cgi marine soldier battle loveaffair antiwar powerrelations mindandsoul 3d Action Adventure Fantasy ScienceFiction In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization. SamWorthington ZoeSaldana SigourneyWeaver JamesCameron'

In [36]:
new_df['tags'] = new_df['tags'].apply(lambda x : x.lower())

C:\Users\rani\AppData\Local\Temp\ipykernel_16476\3679624391.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(lambda x : x.lower())


# Text Vectorization 

In [37]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(max_features=5000,stop_words='english')


In [38]:
vectors = cv.fit_transform(new_df['tags']).toarray() # this will give sparse matrix convert it into array

In [39]:
vectors.shape

(4806, 5000)

In [40]:
vectors[0]

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

we can see the most frequents words using one feature get_feature name

In [41]:
cv.get_feature_names_out()

array(['000', '007', '10', ..., 'zone', 'zoo', 'zooeydeschanel'],
      dtype=object)

# Stemming :
technique to make similar function , same

In [42]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [43]:
import nltk

In [44]:
from nltk.stem.porter import PorterStemmer

ps = PorterStemmer()

In [45]:
def stem(obj):
    y = []
    for i in obj.split():
        y.append(ps.stem(i))
    return " ".join(y)
        

In [46]:
new_df['tags'] = new_df['tags'].apply(stem)

C:\Users\rani\AppData\Local\Temp\ipykernel_16476\3213734980.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(stem)


In [47]:
new_df

,movie_id,title,tags
0,19995,Avatar,cultureclash futur spacewar spacecoloni societ...
1,285,Pirates of the Caribbean: At World's End,ocean drugabus exoticisland eastindiatradingco...
2,206647,Spectre,spi basedonnovel secretag sequel mi6 britishse...
3,49026,The Dark Knight Rises,dccomic crimefight terrorist secretident burgl...
4,49529,John Carter,basedonnovel mar medallion spacetravel princes...
...,...,...,...
4804,9367,El Mariachi,unitedstates–mexicobarri leg arm paperknif gui...
4805,72766,Newlyweds,comedi romanc a newlyw couple' honeymoon is up...
4806,231617,"Signed, Sealed, Delivered",date loveatfirstsight narrat investig team pos...
4807,126186,Shanghai Calling,when ambiti new york attorney sam is sent to s...


In [48]:
cv = CountVectorizer(max_features=5000,stop_words='english')
vector = cv.fit_transform(new_df['tags']).toarray()

In [49]:
cv.get_feature_names_out()

array(['000', '007', '10', ..., 'zone', 'zoo', 'zooeydeschanel'],
      dtype=object)

# Similarity 

we will not be calculating euclidean distance **curse of Dimensionality** because in higher dimension ecludiean distance performance decreases thats why we will be using cosine distance in which we are calculating angle between the two vector and those vector whose angle will be less will be the closest

cos(0) == 1 if both vector are same then angle will be zero value will be one
cos(90) == 0 if they are perpendicular then distance will be zero 

**Distance is inversly propotional to similarity**

In [50]:
from sklearn.metrics.pairwise import cosine_similarity

In [51]:
similarity = cosine_similarity(vector)
similarity[0]

array([1.        , 0.08346223, 0.0860309 , ..., 0.04499213, 0.        ,
       0.        ])

In [52]:
sorted(similarity[0],reverse=True)[1:5]

[0.28676966733820225,
 0.26901379342448517,
 0.2605130246476754,
 0.255608593705383]

In [53]:
similarity[0]

array([1.        , 0.08346223, 0.0860309 , ..., 0.04499213, 0.        ,
       0.        ])

In [54]:
list(enumerate(similarity[0]))

[(0, 1.0000000000000002),
 (1, 0.08346223261119858),
 (2, 0.08603090020146065),
 (3, 0.0734718358370645),
 (4, 0.1892994097121204),
 (5, 0.10838874619051501),
 (6, 0.04024218182927669),
 (7, 0.14673479641335554),
 (8, 0.05923488777590923),
 (9, 0.0967301666813349),
 (10, 0.10259783520851541),
 (11, 0.09464970485606021),
 (12, 0.09037128496931669),
 (13, 0.04499212706658476),
 (14, 0.12824729401064427),
 (15, 0.06282808624375433),
 (16, 0.07894736842105264),
 (17, 0.13977653617040256),
 (18, 0.09493290614465533),
 (19, 0.0830812984794528),
 (20, 0.058038100008800934),
 (21, 0.10968169942141635),
 (22, 0.0662266178532522),
 (23, 0.08740748201220976),
 (24, 0.0533380747062665),
 (25, 0.05101627678885769),
 (26, 0.15389675281277312),
 (27, 0.18693292157876878),
 (28, 0.116543309349613),
 (29, 0.065033247714309),
 (30, 0.06684847767323797),
 (31, 0.15907119074394446),
 (32, 0.08520286456846099),
 (33, 0.09733285267845754),
 (34, 0.0),
 (35, 0.09933992677987831),
 (36, 0.17119059581558146),


In [55]:
sorted(list(enumerate(similarity[0])),reverse=True)

[(4805, 0.0),
 (4804, 0.0),
 (4803, 0.04499212706658476),
 (4802, 0.046829290579084706),
 (4801, 0.019252140716412975),
 (4800, 0.0),
 (4799, 0.052631578947368425),
 (4798, 0.04223886030955117),
 (4797, 0.0),
 (4796, 0.0),
 (4795, 0.0),
 (4794, 0.0),
 (4793, 0.05407380704358751),
 (4792, 0.0),
 (4791, 0.0),
 (4790, 0.0582716546748065),
 (4789, 0.060833032924035954),
 (4788, 0.0),
 (4787, 0.019117977822546817),
 (4786, 0.0),
 (4785, 0.019672236884115842),
 (4784, 0.043355498476206004),
 (4783, 0.0),
 (4782, 0.025649458802128853),
 (4781, 0.05923488777590923),
 (4780, 0.0),
 (4779, 0.0),
 (4778, 0.0),
 (4777, 0.10814761408717502),
 (4776, 0.0),
 (4775, 0.05647824947249051),
 (4774, 0.0),
 (4773, 0.029617443887954616),
 (4772, 0.023918243661746996),
 (4771, 0.039344473768231684),
 (4770, 0.0),
 (4769, 0.0),
 (4768, 0.0),
 (4767, 0.03627381250550058),
 (4766, 0.017195436249022724),
 (4765, 0.0),
 (4764, 0.0),
 (4763, 0.0),
 (4762, 0.0),
 (4761, 0.02742042485535409),
 (4760, 0.0253347295969

In [56]:
sorted(list(enumerate(similarity[0])),reverse=True,key= lambda x:x[1])[1:6]

[(1216, 0.28676966733820225),
 (2409, 0.26901379342448517),
 (3730, 0.2605130246476754),
 (507, 0.255608593705383),
 (539, 0.25038669783359574)]

In [57]:
movie='Avatar'
new_df[new_df['title']==movie].index[0]

0

In [58]:
def recommend(movie):
    movie_index = new_df[new_df['title']==movie].index[0]
    distances = similarity[movie_index]
    movie_list = sorted(list(enumerate(similarity[0])),reverse=True,key= lambda x:x[1])[1:6]
    for i in movie_list:
        print(new_df.iloc[i[0]]['title'])

In [79]:
recommend("Titan A.E")

IndexError: index 0 is out of bounds for axis 0 with size 0

In [60]:
new_df['title'].values

array(['Avatar', "Pirates of the Caribbean: At World's End", 'Spectre',
       ..., 'Signed, Sealed, Delivered', 'Shanghai Calling',
       'My Date with Drew'], dtype=object)

In [61]:
new_df

,movie_id,title,tags
0,19995,Avatar,cultureclash futur spacewar spacecoloni societ...
1,285,Pirates of the Caribbean: At World's End,ocean drugabus exoticisland eastindiatradingco...
2,206647,Spectre,spi basedonnovel secretag sequel mi6 britishse...
3,49026,The Dark Knight Rises,dccomic crimefight terrorist secretident burgl...
4,49529,John Carter,basedonnovel mar medallion spacetravel princes...
...,...,...,...
4804,9367,El Mariachi,unitedstates–mexicobarri leg arm paperknif gui...
4805,72766,Newlyweds,comedi romanc a newlyw couple' honeymoon is up...
4806,231617,"Signed, Sealed, Delivered",date loveatfirstsight narrat investig team pos...
4807,126186,Shanghai Calling,when ambiti new york attorney sam is sent to s...


In [62]:
import pickle

In [63]:
pickle.dump(new_df,open('movies.pkl','wb')) #write binary mode

In [65]:
pickle.dump(similarity,open('similarity.pkl','wb'))

In [72]:
x={"adult":False,"backdrop_path":"/hZkgoQYus5vegHoetLkCJzb17zJ.jpg","belongs_to_collection":None,"budget":63000000,"genres":[{"id":18,"name":"Drama"},{"id":53,"name":"Thriller"},{"id":35,"name":"Comedy"}],"homepage":"http://www.foxmovies.com/movies/fight-club","id":550,"imdb_id":"tt0137523","original_language":"en","original_title":"Fight Club","overview":"A ticking-time-bomb insomniac and a slippery soap salesman channel primal male aggression into a shocking new form of therapy. Their concept catches on, with underground \"fight clubs\" forming in every town, until an eccentric gets in the way and ignites an out-of-control spiral toward oblivion.","popularity":97.396,"poster_path":"/pB8BM7pdSp6B6Ih7QZ4DrQ3PmJK.jpg","production_companies":[{"id":508,"logo_path":"/7cxRWzi4LsVm4Utfpr1hfARNurT.png","name":"Regency Enterprises","origin_country":"US"},{"id":711,"logo_path":"/tEiIH5QesdheJmDAqQwvtN60727.png","name":"Fox 2000 Pictures","origin_country":"US"},{"id":4700,"logo_path":"/A32wmjrs9Psf4zw0uaixF0GXfxq.png","name":"The Linson Company","origin_country":"US"},{"id":20555,"logo_path":"/hD8yEGUBlHOcfHYbujp71vD8gZp.png","name":"Taurus Film","origin_country":"DE"},{"id":54051,"logo_path":None,"name":"Atman Entertainment","origin_country":""},{"id":54052,"logo_path":None,"name":"Knickerbocker Films","origin_country":"US"}],"production_countries":[{"iso_3166_1":"DE","name":"Germany"},{"iso_3166_1":"US","name":"United States of America"}],"release_date":"1999-10-15","revenue":100853753,"runtime":139,"spoken_languages":[{"english_name":"English","iso_639_1":"en","name":"English"}],"status":"Released","tagline":"Mischief. Mayhem. Soap.","title":"Fight Club","video":False,"vote_average":8.428,"vote_count":25236}

In [75]:
x['poster_path']

'/pB8BM7pdSp6B6Ih7QZ4DrQ3PmJK.jpg'

In [76]:
https://api.themoviedb.org/3/movie/{}?api_key=2234cda012e52526ee792fda591639f4&language=en-US

SyntaxError: invalid syntax (2088713073.py, line 1)

In [77]:
2234cda012e52526ee792fda591639f4

SyntaxError: invalid syntax (317371008.py, line 1)

In [ ]:

https://api.themoviedb.org/3/movie/550?api_key=2234cda012e52526ee792fda591639f4&language=en-US